# Clustering to predict missing fields


In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
from itertools import compress
from sklearn.ensemble import ExtraTreesRegressor
from scipy.stats import norm
from scipy import spatial
from collections import Counter
import operator

%matplotlib inline

pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999


# Setting up the data

In [2]:
train_df = pd.read_pickle("C:\\Users\\lakshay\\Documents\\zes\\zestims\\data\\input\\train_df_full.pkl")
all_null_ixs = pd.isnull(train_df["longitude"]).nonzero()[0]
train_df = train_df.drop(all_null_ixs)

# Using KDTree to find nearest neighbors and classify the fields

In [3]:
# Create Tree to be used for clustering applications
tree = spatial.KDTree(zip(train_df["longitude"].values,train_df["latitude"].values))

In [7]:
# class to classify fields based on clustering
# Classify region ids using nearest region center method

class cluster(object):
    
    def __init__(self,feature,tree=tree):
        self.feature = feature

    def classify(self,x,df,k):
        [m,n] = tree.query(x,k=k)
        near_ids = df[self.feature].iloc[n].values
        near_ids = near_ids[~pd.isnull(near_ids)]
        c = Counter(near_ids)
        try:
            output = max(c.iteritems(),key=operator.itemgetter(1))[0]
            #near_ids.tolist()
            #a = [int(i) for i in near_ids]
            #counts = np.bincount(a)
            return output
        except ValueError:
            pass


In [18]:
# Features to be filled from clustering
# regionidcity,regionidzip, regionidneighborhood already completed
# propertyzoningdesc - too many - 1027113, nothing happens even after 3 iterations
# propertycountylandusecode - 841 still missing
# propertylandusetypeid is already completed

f = ("propertylandusetypeid","propertylandusetypeid")

for feature in f:
    print feature
    print feature
    f_cluster = cluster(feature)
    train_df_feature = train_df[[feature,'longitude','latitude']]
    tdf_feature_no_null = train_df_feature.dropna()
    tdf_feature_all_null = train_df_feature[~train_df_feature.isin(tdf_feature_no_null).all(1)]
    
    iteration = [10,100,500,1000,1500,2000,2500,5000]
    t = 0
    print("Number of NULL values at the start: %g" %train_df_feature[feature].isnull().sum(axis=0))
    while train_df_feature[feature].isnull().sum(axis=0) > 0:
        for j in list(tdf_feature_all_null.index.values):
            x = np.array([train_df_feature.loc[j]["longitude"],train_df_feature.loc[j]["latitude"]])
            train_df_feature.loc[j][feature] = f_cluster.classify(x,train_df_feature,iteration[t])

        # Update dataframes 
        tdf_feature_no_null = train_df_feature.dropna()
        tdf_feature_all_null = train_df_feature[~train_df_feature.isin(tdf_feature_no_null).all(1)]
        print train_df_feature[feature].isnull().sum(axis=0)     
        t += 1
        
    train_df_feature.to_pickle("C:\\Users\\lakshay\\Documents\\zes\\zestims\\data\\input\\train_df_"+ \
                               feature+"_completed_all_data.pkl")


propertylandusetypeid
propertylandusetypeid
Number of NULL values at the start: 0
propertylandusetypeid
propertylandusetypeid
Number of NULL values at the start: 0


In [11]:
del f

In [19]:
train_df_feature.isnull().sum(axis=0)

propertylandusetypeid    0
longitude                0
latitude                 0
dtype: int64

In [4]:
train_df.columns.values

array(['parcelid', 'logerror', 'transactiondate', 'airconditioningtypeid',
       'architecturalstyletypeid', 'basementsqft', 'bathroomcnt',
       'bedroomcnt', 'buildingclasstypeid', 'buildingqualitytypeid',
       'calculatedbathnbr', 'decktypeid', 'finishedfloor1squarefeet',
       'calculatedfinishedsquarefeet', 'finishedsquarefeet12',
       'finishedsquarefeet13', 'finishedsquarefeet15',
       'finishedsquarefeet50', 'finishedsquarefeet6', 'fips',
       'fireplacecnt', 'fullbathcnt', 'garagecarcnt', 'garagetotalsqft',
       'hashottuborspa', 'heatingorsystemtypeid', 'latitude', 'longitude',
       'lotsizesquarefeet', 'poolcnt', 'poolsizesum', 'pooltypeid10',
       'pooltypeid2', 'pooltypeid7', 'propertycountylandusecode',
       'propertylandusetypeid', 'propertyzoningdesc',
       'rawcensustractandblock', 'regionidcity', 'regionidcounty',
       'regionidneighborhood', 'regionidzip', 'roomcnt', 'storytypeid',
       'threequarterbathnbr', 'typeconstructiontypeid', 'unitcn